In [148]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
import pickle

In [149]:
#data_preprocessing

def data_preprocessing(dataset:pd.DataFrame):
    '''
    dataset contain this columns:
        'gender':               ['Female' 'Male']
        'SeniorCitizen':        [0 1]
        'Partner':              ['Yes' 'No']
        'Dependents':           ['No' 'Yes']
        'tenure':               int
        'PhoneService':         ['No' 'Yes']
        'MultipleLines':        ['No phone service' 'No' 'Yes']
        'InternetService':      ['DSL' 'Fiber optic' 'No']
        'OnlineSecurity':       ['No' 'Yes' 'No internet service']
        'OnlineBackup':         ['Yes' 'No' 'No internet service']
        'DeviceProtection':     ['No' 'Yes' 'No internet service']
        'TechSupport':          ['No' 'Yes' 'No internet service']
        'StreamingTV':          ['No' 'Yes' 'No internet service']
        'StreamingMovies':      ['No' 'Yes' 'No internet service']
        'Contract':             ['Month-to-month' 'One year' 'Two year']
        'PaperlessBilling':     ['Yes' 'No']
        'PaymentMethod':        ['Electronic check' 'Mailed check' 'Bank transfer (automatic)' 'Credit card (automatic)']
        'MonthlyCharges':       float
        'TotalCharges':         float
        'Churn':                ['No' 'Yes']   // Not necessary
    '''
    df = dataset.copy()

    # Prepare string variables
    df.columns = df.columns.str.lower().str.replace(' ','_')
    string_columns = list(df.dtypes[df.dtypes=='O'].index)
    for col in string_columns:
        df[col] = df[col].str.lower().str.replace(' ','_')

    # drop customerid
    if 'customerid' in df.columns:
        del df['customerid']
        
    # Prepare numeric columns type
    if 'tenure' in df.columns:
        df['tenure'] = pd.to_numeric(df['tenure'], errors='coerce')
        df['tenure'] = df['tenure'].fillna(0)
    else:
        raise Exception("!!! Tenure column didn't exist !!!")

    if 'monthlycharges' in df.columns:
        df['monthlycharges'] = pd.to_numeric(df['monthlycharges'], errors='coerce')
        df['monthlycharges'] = df['monthlycharges'].fillna(0)
    else:
        raise Exception("!!! MonthlyCharges column didn't exist !!!")

    if 'totalcharges' in df.columns:
        df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
        df['totalcharges'] = df['totalcharges'].fillna(0)
    else:
        raise Exception("!!! TotalCharges column didn't exist !!!")
        
    # Make dependent variable numeric
    if 'churn' in df.columns:
        df.churn = (df.churn == 'yes').astype(int)
    
    return df

In [150]:
# FeatureEngineering

def FeatureEngineering(dataset:pd.DataFrame, categorical_variables:list, numerical_variables:list, isTrain:bool):
    """
    dataset: pd.Dataframe
    categorical_variables: list of important categorical columns that we want to use in pur model
    numerical_variables: list of important numerical columns that we want to use in pur model
    isTrain:    if you want to train a model, set it 'True'
                if you want to use for test or deployment set it 'False'
    """
    df = dataset.copy()
    categorical_important = categorical_variables.copy()
    numerical_important = numerical_variables.copy()
    
    df = df[numerical_important + categorical_important]
    
    # Dummy variables
    if isTrain:
        OHE = make_column_transformer((OneHotEncoder(), categorical_important ),
                                            remainder='passthrough',
                                            verbose_feature_names_out=False)
        ohe = OHE.fit_transform(df)
        pickle.dump(ohe, open('OneHotEncoder.pkl','wb'))
        df = pd.DataFrame(ohe, columns=OHE.get_feature_names_out())

        # we Save OHE.get_feature_names_out() to use in test process or deployment in future
        with open("OHE_feature_names_out.txt", "w") as file:
            for item in OHE.get_feature_names_out():
                file.write('%s\n' %item)
        file.close()
    else:
        ohe = pickle.load(open('OneHotEncoder.pkl','rb'))
        OHE_feature_names_out=[]
        with open('OHE_feature_names_out.txt') as file:
            OHE_feature_names_out = file.readlines()
        file.close()
        OHE_feature_names_out = list(map(lambda x: x[:-1], OHE_feature_names_out))

        df = pd.DataFrame(ohe, columns=OHE_feature_names_out)
        
    # Feature scaling
    if isTrain:
        scaler = StandardScaler()
        scaler.fit(df)
        df = scaler.transform(df)
        df = pd.DataFrame(df, columns=scaler.feature_names_in_)
        pickle.dump(scaler, open('scaler.pkl','wb'))
    else:
        scaler = pickle.load(open('scaler.pkl','rb'))
        df = scaler.transform(df)
        df = pd.DataFrame(df, columns=scaler.feature_names_in_)

    return df

In [151]:
df = pd.read_csv('Churn_prediction.csv')
df.drop_duplicates(inplace=True)

<h2>Split dataset</h2>

In [152]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.loc[:,df.columns!='Churn'],
                                                    df['Churn'],
                                                    test_size=0.2,
                                                    random_state=666)

X_train.reset_index(inplace=True,drop=True)
X_test.reset_index(inplace=True,drop=True)

y_train = [1 if x == 'Yes' else 0 for x in y_train]
y_test = [1 if x == 'Yes' else 0 for x in y_test]


In [153]:
print("Length X_train:{0}, lengthy_train:{1}".format(len(X_train),len(y_train)))
print("Length X_test:{0}, lengthy_test:{1}".format(len(X_test),len(y_test)))

Length X_train:5634, lengthy_train:5634
Length X_test:1409, lengthy_test:1409


In [154]:
categorical_important = ['contract', 'onlinesecurity', 'techsupport', 'internetservice']
numerical_important = ['tenure', 'monthlycharges', 'totalcharges']

X_train = data_preprocessing(X_train)
X_train = FeatureEngineering(X_train,
                              categorical_variables=categorical_important,
                              numerical_variables=numerical_important,
                              isTrain=True)


<h2>1) Logistic Regression</h2>

In [155]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
clf_LogisticRegression = LogisticRegression(random_state=0)

scores = cross_validate(clf_LogisticRegression, X_train, y_train, cv=6,
                        scoring=('roc_auc'),
                        return_train_score=True)
print(scores['test_score'].mean())

0.8367262420328858


<h2>2) K Nearest Neighbors</h2>

In [156]:
from sklearn.neighbors import KNeighborsClassifier
clf_KNN = KNeighborsClassifier()

scores = cross_validate(clf_KNN, X_train, y_train, cv=6,
                        scoring=('roc_auc'),
                        return_train_score=True)
print(scores['test_score'].mean())


0.7796812937086126


<h2>3) Naive Bayes</h2>

In [157]:
from sklearn.naive_bayes import GaussianNB
clf_NB = GaussianNB()

scores = cross_validate(clf_NB, X_train, y_train, cv=6,
                        scoring=('roc_auc'),
                        return_train_score=True)
print(scores['test_score'].mean())


0.8254343639998684


<h2>4) Support Vector machine for Classification (SVC)</h2>

In [158]:
from sklearn.svm import SVC
clf_SVC = SVC(kernel='linear', random_state=6)

scores = cross_validate(clf_SVC, X_train, y_train, cv=6,
                        scoring=('roc_auc'),
                        return_train_score=True)
print(scores['test_score'].mean())
# kernel
#  -sigmoid   0.7217249049598338
#  -poly      0.8158020900887415
#  -rbf       0.7720218063211098
#  -linear    0.8092300188309788


0.8092300188309788


<h2>5) Desicion Tree Classification</h2>

In [159]:
from calendar import c
from sklearn.tree import DecisionTreeClassifier
clf_DT = DecisionTreeClassifier(criterion='entropy', random_state=6)

scores = cross_validate(clf_DT, X_train, y_train, cv=6,
                        scoring=('roc_auc'),
                        return_train_score=True)
print(scores['test_score'].mean())
# criterion
#   -gini      0.666598842172868
#   -entropy   0.6673704556237601

0.6673704556237601


<h2>6) Random Forest</h2>

In [160]:
from sklearn.ensemble import RandomForestClassifier

clf_RF = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=6)

scores = cross_validate(clf_RF, X_train, y_train, cv=6,
                        scoring=('roc_auc'),
                        return_train_score=True)
print(scores['test_score'].mean())
# criterion
#   -gini       0.7940777923624925    
#   -entropy    0.7955202892868366  

0.7955202892868366


<h2>7) XGBoost</h2>

In [161]:
from xgboost import XGBClassifier

clf_xgboost = XGBClassifier()

scores = cross_validate(clf_xgboost, X_train, y_train, cv=6,
                        scoring=('roc_auc'),
                        return_train_score=True)
print(scores['test_score'].mean())

0.8171957479568427


<h2>8) Gradiant Boosting model</h2>

In [162]:
from sklearn.ensemble import GradientBoostingClassifier

clf_GB = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1, max_depth=3)

scores = cross_validate(clf_GB, X_train, y_train, cv=6,
                        scoring=('roc_auc'),
                        return_train_score=True)
print(scores['test_score'].mean())

0.8405381208558101


<h2>TEST</h2>

In [163]:
print("Length X_train:{0}, lengthy_train:{1}".format(len(X_train),len(y_train)))
print("Length X_test:{0}, lengthy_test:{1}".format(len(X_test),len(y_test)))

Length X_train:5634, lengthy_train:5634
Length X_test:1409, lengthy_test:1409


In [164]:
X_test = data_preprocessing(X_test)
print(1)
print("Length X_train:{0}, lengthy_train:{1}".format(len(X_train),len(y_train)))
print("Length X_test:{0}, lengthy_test:{1}".format(len(X_test),len(y_test)))
X_test = FeatureEngineering(X_test,
                            categorical_variables=categorical_important,
                            numerical_variables=numerical_important,
                            isTrain=False)
print(2)
print("Length X_train:{0}, lengthy_train:{1}".format(len(X_train),len(y_train)))
print("Length X_test:{0}, lengthy_test:{1}".format(len(X_test),len(y_test)))
# model
clf_LogisticRegression = LogisticRegression(random_state=0)
clf_LogisticRegression.fit(X_train, y_train)
print(3)
print("Length X_train:{0}, lengthy_train:{1}".format(len(X_train),len(y_train)))
print("Length X_test:{0}, lengthy_test:{1}".format(len(X_test),len(y_test)))
# clf_GB = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1, max_depth=3)
# clf_GB.fit(df_train, y_train)



pred = clf_LogisticRegression.predict(X_test)

from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, pred)
# 0.6610603450428183

1
Length X_train:5634, lengthy_train:5634
Length X_test:1409, lengthy_test:1409
2
Length X_train:5634, lengthy_train:5634
Length X_test:5634, lengthy_test:1409
3
Length X_train:5634, lengthy_train:5634
Length X_test:5634, lengthy_test:1409


ValueError: Found input variables with inconsistent numbers of samples: [1409, 5634]